In [180]:
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2016)
folds = kf.split(range(len(y)))
S_train_i = np.zeros(len(y))
print len(y)
for j, (train_idx, test_idx) in enumerate(folds):
    print (train_idx, test_idx)

30396
(array([    1,     2,     3, ..., 30393, 30394, 30395]), array([    0,     5,    10, ..., 30385, 30388, 30392]))
(array([    0,     1,     2, ..., 30391, 30392, 30395]), array([    9,    16,    18, ..., 30382, 30393, 30394]))
(array([    0,     1,     2, ..., 30393, 30394, 30395]), array([    4,     6,     7, ..., 30377, 30384, 30389]))
(array([    0,     4,     5, ..., 30393, 30394, 30395]), array([    1,     2,     3, ..., 30378, 30390, 30391]))
(array([    0,     1,     2, ..., 30392, 30393, 30394]), array([   15,    20,    26, ..., 30386, 30387, 30395]))


In [155]:
ID = np.array([1,2,3])

In [154]:
df = pd.DataFrame({'id':[1,2], 'y':[4,5]})

In [157]:
df2 = pd.DataFrame(ID, columns=['id'])

In [158]:
df2.merge(df, on='id', how='left')

,id,y
0,1,4.0
1,2,5.0
2,3,NaN


In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import model_selection, preprocessing
from sklearn import ensemble   
from sklearn import datasets   
from sklearn.utils import shuffle   
import xgboost as xgb
color = sns.color_palette()

%matplotlib inline

pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.max_columns', 500)

In [85]:
from stacking import *

In [83]:
import stacking

In [84]:
reload(stacking)

<module 'stacking' from 'stacking.py'>

In [4]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

In [5]:
def logrmse(predictions, targets):
    return rmse(np.log(predictions), np.log(targets))

# bruno

In [6]:
import bruno

In [72]:
reload(bruno)

<module 'bruno' from 'bruno.py'>

In [73]:
xgb_params = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 0
}
num_boost_rounds = 420  # From Bruno's original CV, I think
bruno_xgb = XgbModel(xgb_params, 1000)

# gunja

In [9]:
import gunja

In [10]:
reload(gunja)

<module 'gunja' from 'gunja.pyc'>

In [74]:
xgb_params = {
    'eta': 0.05,
    'max_depth': 6,
    'subsample': 0.6,
    'colsample_bytree': 1,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 0
}
num_boost_rounds = 422
gunja_xgb = XgbModel(xgb_params, 1000)

# louis

In [12]:
import louis

In [13]:
reload(louis)

<module 'louis' from 'louis.pyc'>

In [75]:
xgb_params = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 0
}
num_boost_rounds = 385  # This was the CV output, as earlier version shows
louis_xgb = XgbModel(xgb_params, 1000)

# stacking

In [57]:
df_train = pd.read_csv('../input/train.csv')
id_train = df_train['id'].values
y = df_train['price_doc'].values

In [86]:
class DataResolver(object):
    def __init__(self):
        self.__time = -1
    
    def next(self):
        self.__time = self.__time + 1
        if (self.__time >= 2):
            return louis.resolver()
        if (self.__time >= 1):
            return gunja.resolver()
        return bruno.resolver()

In [87]:
base_models = []
base_models.append(bruno_xgb)
base_models.append(gunja_xgb)
base_models.append(louis_xgb)
stacking = Stacking(5, base_models, DataResolver(), logrmse)

In [88]:
s_train, s_test = stacking.fit(id_train, 7662)#38132 30471

model 0
fold 0
[0]	train-rmse:7.94147e+06	val-rmse:8.05408e+06
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 20 rounds.
[20]	train-rmse:3.81419e+06	val-rmse:4.13389e+06
[40]	train-rmse:2.65213e+06	val-rmse:3.14326e+06
[60]	train-rmse:2.31299e+06	val-rmse:2.90927e+06
[80]	train-rmse:2.18849e+06	val-rmse:2.84577e+06
[100]	train-rmse:2.12534e+06	val-rmse:2.80723e+06
[120]	train-rmse:2.07491e+06	val-rmse:2.78252e+06
[140]	train-rmse:2.03595e+06	val-rmse:2.76697e+06
[160]	train-rmse:1.99745e+06	val-rmse:2.76209e+06
[180]	train-rmse:1.96417e+06	val-rmse:2.75261e+06
[200]	train-rmse:1.93173e+06	val-rmse:2.74295e+06
[220]	train-rmse:1.90582e+06	val-rmse:2.73929e+06
[240]	train-rmse:1.88328e+06	val-rmse:2.73455e+06
[260]	train-rmse:1.85779e+06	val-rmse:2.73073e+06
[280]	train-rmse:1.83153e+06	val-rmse:2.72764e+06
[300]	train-rmse:1.80862e+06	val-rmse:2.72574e+06
[320]	train-rmse:1.79103e+06	val-rmse:2.72475e+06


d:\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: RuntimeWarning: invalid value encountered in log
  from ipykernel import kernelapp as app


 nan val: 0.452754585736
fold 2
[0]	train-rmse:7.96103e+06	val-rmse:7.91597e+06
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 20 rounds.
[20]	train-rmse:3.8464e+06	val-rmse:3.93034e+06
[40]	train-rmse:2.67898e+06	val-rmse:2.94584e+06
[60]	train-rmse:2.35306e+06	val-rmse:2.73964e+06
[80]	train-rmse:2.23429e+06	val-rmse:2.67918e+06
[100]	train-rmse:2.16653e+06	val-rmse:2.65197e+06
[120]	train-rmse:2.10614e+06	val-rmse:2.63054e+06
[140]	train-rmse:2.06556e+06	val-rmse:2.62585e+06
[160]	train-rmse:2.02923e+06	val-rmse:2.61531e+06
[180]	train-rmse:1.99709e+06	val-rmse:2.6067e+06
[200]	train-rmse:1.9622e+06	val-rmse:2.60232e+06
[220]	train-rmse:1.93737e+06	val-rmse:2.59722e+06
[240]	train-rmse:1.91047e+06	val-rmse:2.59282e+06
[260]	train-rmse:1.88963e+06	val-rmse:2.59338e+06
[280]	train-rmse:1.86432e+06	val-rmse:2.59133e+06
[300]	train-rmse:1.84158e+06	val-rmse:2.58971e+06
[320]	train-rmse:1.8208e+06	val-rmse

In [89]:
pd.DataFrame(s_train).to_csv('../stacking/5_3/5_3_train.csv', index=False)

In [90]:
pd.DataFrame(s_test).to_csv('../stacking/5_3/5_3_test.csv', index=False)

In [49]:
s_train

array([[  3804734.  ,   3631560.25,   5081369.  ],
       [  5614989.5 ,   4439626.5 ,   5466862.  ],
       [  4921093.  ,   4305226.5 ,   4645399.5 ],
       ..., 
       [  5405239.  ,   6122101.  ,   5312807.  ],
       [ 11368580.  ,  12399014.  ,  10650168.  ],
       [  6148419.  ,   6521823.5 ,   6236720.5 ]])

In [95]:
s_train[:25000].shape

(25000L, 3L)

In [92]:
s_test

array([[ 5446768.  ,  5541906.8 ,  5346134.6 ],
       [ 8194974.4 ,  8371003.4 ,  8025939.1 ],
       [ 5307726.5 ,  5458503.8 ,  5341008.3 ],
       ..., 
       [ 5575524.1 ,  3963156.15,  4804156.1 ],
       [ 5085413.6 ,  4385851.35,  5267258.3 ],
       [ 9568489.9 ,  8166173.1 ,  8489979.6 ]])

In [104]:
xgb_params = {
    'eta': 0.05,
    'max_depth': 6,
    'subsample': 0.6,
    'colsample_bytree': 1,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}

dtrain = xgb.DMatrix(s_train, y)
#dtrain = xgb.DMatrix(s_train[:25000], y[:25000])
dval = xgb.DMatrix(s_train[25000:], y[25000:])
dtest = xgb.DMatrix(s_test)


num_boost_rounds = 1000
model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round=200, evals={(dtrain,'train')})
#model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round=num_boost_rounds, evals=[(dtrain,'train'), (dval,'val')], early_stopping_rounds=20, verbose_eval=20)

y_predict = model.predict(dtest)

[0]	train-rmse:8.19032e+06
[1]	train-rmse:7.82544e+06
[2]	train-rmse:7.47856e+06
[3]	train-rmse:7.15087e+06
[4]	train-rmse:6.84183e+06
[5]	train-rmse:6.54782e+06
[6]	train-rmse:6.27054e+06
[7]	train-rmse:6.01081e+06
[8]	train-rmse:5.76534e+06
[9]	train-rmse:5.53002e+06
[10]	train-rmse:5.31225e+06
[11]	train-rmse:5.10832e+06
[12]	train-rmse:4.91379e+06
[13]	train-rmse:4.72867e+06
[14]	train-rmse:4.55757e+06
[15]	train-rmse:4.3961e+06
[16]	train-rmse:4.24577e+06
[17]	train-rmse:4.10588e+06
[18]	train-rmse:3.97484e+06
[19]	train-rmse:3.85007e+06
[20]	train-rmse:3.73837e+06
[21]	train-rmse:3.62969e+06
[22]	train-rmse:3.52903e+06
[23]	train-rmse:3.43398e+06
[24]	train-rmse:3.34586e+06
[25]	train-rmse:3.26544e+06
[26]	train-rmse:3.19046e+06
[27]	train-rmse:3.1186e+06
[28]	train-rmse:3.05397e+06
[29]	train-rmse:2.99224e+06
[30]	train-rmse:2.93796e+06
[31]	train-rmse:2.88646e+06
[32]	train-rmse:2.83955e+06
[33]	train-rmse:2.79572e+06
[34]	train-rmse:2.7551e+06
[35]	train-rmse:2.7159e+06
[36]	t

In [61]:
df_test = pd.read_csv('../input/test.csv')
id_test = df_test.id

In [105]:
stacking_output = pd.DataFrame({'id': id_test, 'price_doc': y_predict })

In [106]:
stacking_output.to_csv('../stacking.csv', index=False)